In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, hour, month
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder, StandardScaler
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator

# Khởi tạo Spark Session
spark = SparkSession.builder \
    .appName("WeatherPredictionGBT") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

# In URL của Web UI để theo dõi
print("Spark Web UI running at:", spark.sparkContext.uiWebUrl)

# Đọc dữ liệu
file_path = "C:/bigdata/weatherHistory.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Thêm đặc trưng thời gian
df = df.withColumn("Hour", hour(col("Formatted Date")))
df = df.withColumn("Month", month(col("Formatted Date")))

# Mã hóa cột Summary (loại thời tiết)
indexer = StringIndexer(inputCol="Summary", outputCol="SummaryIndex")
df = indexer.fit(df).transform(df)
encoder = OneHotEncoder(inputCols=["SummaryIndex"], outputCols=["SummaryVec"])
df = encoder.fit(df).transform(df)

# Chọn đặc trưng
feature_cols = ["Humidity", "Wind Speed (km/h)", "Wind Bearing (degrees)", 
                "Visibility (km)", "Pressure (millibars)", "SummaryVec", "Hour", "Month"]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
final_df = assembler.transform(df).select(col("features"), col("Temperature (C)").alias("label"))

# Chuẩn hóa dữ liệu
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=True)
scaler_model = scaler.fit(final_df)
final_df = scaler_model.transform(final_df).select(col("scaledFeatures").alias("features"), "label")

# Chia dữ liệu thành tập huấn luyện và kiểm tra
train_data, test_data = final_df.randomSplit([0.8, 0.2], seed=42)
print(f"Training data rows: {train_data.count()}")
print(f"Test data rows: {test_data.count()}")

# Huấn luyện GBTRegressor
spark.sparkContext.setJobGroup("GBT Training", "Training Gradient Boosted Trees")
gbt = GBTRegressor(featuresCol="features", labelCol="label", maxIter=50, maxDepth=5)
gbt_model = gbt.fit(train_data)

# Dự đoán trên tập kiểm tra
spark.sparkContext.setJobGroup("Prediction", "Making predictions with GBT")
predictions = gbt_model.transform(test_data)
predictions.select("features", "label", "prediction").show(5)

# Đánh giá mô hình
evaluator_rmse = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
evaluator_r2 = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="r2")
rmse = evaluator_rmse.evaluate(predictions)
r2 = evaluator_r2.evaluate(predictions)
print(f"GBT RMSE: {rmse}")
print(f"GBT R2: {r2}")
# Dừng Spark Session
input("Press Enter to stop Spark and close Web UI...")
spark.stop()

Spark Web UI running at: http://DESKTOP-MGIJLCC:4042
Training data rows: 77349
Test data rows: 19104
+--------------------+------------------+------------------+
|            features|             label|        prediction|
+--------------------+------------------+------------------+
|[-3.0433858407814...|38.788888888888884|34.241805342309796|
|[-2.9922278120572...| 21.20555555555556|23.600904225477738|
|[-2.9922278120572...|38.983333333333334|33.496929866584736|
|[-2.9410697833330...| 37.89444444444444|35.399644964992135|
|[-2.8899117546088...| 38.71666666666667|34.319523333269565|
+--------------------+------------------+------------------+
only showing top 5 rows

GBT RMSE: 3.3797962304839406
GBT R2: 0.8755016620522316
